In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

In [18]:
data = pd.read_csv('/Users/liuyang/Desktop/High_07.csv')
data1 = pd.read_csv('/Users/liuyang/Desktop/High_08.csv')
data2 = pd.read_csv('/Users/liuyang/Desktop/High_06.csv')

In [19]:
final_data = pd.concat([data,data1,data2], ignore_index = True)

In [20]:
def inf_removal(data_06):
  data_06['15_avg_duration'] = data_06['15_avg_duration'].replace([np.inf, -np.inf],np.nan)
  data_06['15_avg_duration'].fillna(0)
  data_06['15_avg_price'] = data_06['15_avg_price'].replace([np.inf, -np.inf],np.nan)
  data_06['15_avg_price'].fillna(0)
  return data_06

In [21]:
cleaned_data = inf_removal(final_data)

In [22]:
def labels_train_extraction(data_06):
  labels = data_06['15_demand']
  dataset = data_06[['weekday', 'hour', 'minute', 'day','LocationID', 'longitude', 'latitude']]
  return labels, dataset

In [23]:
labels, dataset = labels_train_extraction(cleaned_data)

In [24]:
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size = 0.2)

In [25]:
rf_model = RandomForestRegressor(n_estimators = 20, criterion = 'mse', bootstrap = True, min_samples_leaf = 1)

In [26]:
rf_model.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=20,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [27]:
feat_labels = dataset.columns[0:]
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(x_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) hour                           0.514877
 2) weekday                        0.123796
 3) LocationID                     0.093624
 4) longitude                      0.087557
 5) day                            0.084120
 6) latitude                       0.053871
 7) minute                         0.042155


In [28]:
result = rf_model.predict(x_test)

In [29]:
def evaluation_methods(predict_value, true_value):
  Differences = list()
  for i, j in zip(predict_value, true_value):
    a = abs(i-j)
    Differences.append(a)
  relative_accuracy = np.sum(result)/np.sum(true_value)
  return relative_accuracy
result_final = evaluation_methods(result,y_test)
print(result_final)

1.0001947102066853


In [30]:
MAE = metrics.mean_absolute_error(y_test, result)
MSE = metrics.mean_squared_error(y_test,result)
print(f'Mean squared_error is {MSE}\n Mean Absolute error is {MAE}')

Mean squared_error is 9.632806593424739
 Mean Absolute error is 2.1760635963877593


In [123]:
visualize = pd.DataFrame(abs(result-y_test))

In [139]:
test_index = visualize[visualize['15_demand']>5].index

In [157]:
visualize[visualize['15_demand']>50]

,15_demand
433976,55.250
41835,66.550
204303,58.250
173895,53.750
129495,83.425
...,...
234412,61.475
283370,56.975
369850,62.450
221517,57.575
